In [ ]:
from scapy.all import *

import random
import string
import time
from datetime import datetime
from tqdm import tqdm
def randStr(chars = string.ascii_uppercase + string.digits, lower=0, upper=100):
    if upper<0:
        upper = 0
    padding = random.randint(lower, upper)
    return ''.join(random.choice(chars) for _ in range(padding))



filters = []
#filters = ['DHCP', 'DNS'] # protocols to exclude

max_len = 1500 # can choose: 1500 (typical MTU) or 65000 (maximum possible length)
trigger = 100000 # update value
min_padding = 1
small_padding = 100 
large_padding = 1024 

min_size = 0 
large_size = 100

import platform
if platform.system() == 'Windows':
    fs = "\\"

In [ ]:
def byte_udp(input_file, output_file, lower=0, upper=100, debug = False, max_size = 65000):
    start = time.time()
    print("Start Time\n{}".format(datetime.now()))

    input_pcap_size = int(os.path.getsize(input_file) / (1024*1024))
    print("Reading {}MB from {}".format(input_pcap_size, input_file))
    read = 0
    manipulated = 0
    with PcapReader(input_file) as pr:
        print("Manipulating...")
        new_packets = list([])
        for pkt in pr:
            read += 1
            if ((read%trigger) == 0):
                print("...read {} packets...".format(int(read)))
            if 'error' in pkt.summary():
                new_packets.append(pkt)

            elif 'TCP' in pkt.summary():
                # TCP PACKETS are EXCLUDED
                new_packets.append(pkt)

            elif 'UDP' in pkt.summary():
                #check for filters
                if len(filters)>0:
                    if any(f in pkt.summary() for f in filters):
                        continue
                manipulated += 1
                padding = randStr(lower=lower, upper=min((max_size - pkt.len), upper))



                new_pkt = (pkt['Ether'] / padding)

                if 'IPv6' in pkt.summary():
                    del new_pkt['IPv6'].len
                    del new_pkt['IPv6'].chksum
                else:
                    del new_pkt['IP'].len
                    del new_pkt['IP'].chksum
                del new_pkt['UDP'].len
                del new_pkt['UDP'].chksum
                new_pkt = Ether(new_pkt.build())
                new_pkt.time = pkt.time
                new_packets.append(new_pkt)
                
                if debug:
                    if pkt.len>10:
                        print("Old Pkt Len: {}\tUDP len: {}".format(pkt.len, pkt['UDP'].len))
                        print("New Pkt Len: {}\tUDP len: {}".format(new_pkt.len, new_pkt['UDP'].len))
                        print("Padding length: {}".format(len(padding)))

            else:
                new_packets.append(pkt)

    print("Total packets read: {}".format(read))


    print("Writing {} packets (of which {} manipulated)...".format(len(new_packets), manipulated))
    wrpcap(output_file, new_packets)

    end = time.time() - start
    print("End Time:\n{}".format(datetime.now()))
    print("(total running time: {:5f}s)".format(end))

    output_pcap_size = int(os.path.getsize(output_file) / (1024*1024))
    diff = output_pcap_size - input_pcap_size
    print("Wrote a total of {}MB (diff: {}MB) to file: {}".format(output_pcap_size, diff, output_file))
    return new_packets